In [17]:
import pandas as pd

In [18]:
df = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 신호등 관련 정보.csv",encoding='CP949')

/Users/chanju/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
df.head(5)

,부착대관리번호,상태 (공통),부착방식,부착대길이,고가 (공통),부착대방향,신호등수량,배면등수량,신호등종류,배면등종류,...,작업구분 (공통),표출구분 (공통),신규정규화ID,공사관리번호,부착대관리번호.1,이력ID,위치정보,X좌표,Y좌표,공사형태 (공통)
0,03-0000034286,001,003,009,001,195,2,0,003,002,...,4,001,,2004-1101-152,03-034286,88931,,202650.00560,542705.46253,
1,03-0000044336,001,003,011,001,102,3,0,003,002,...,2,001,,2004-1101-152,03-044336,88932,,202714.48048,542715.77569,
2,03-0000044307,001,002,999,001,312,1,0,007,,...,1,002,,2008-0201-006,03-044307,88933,,203129.34784,541838.75856,
3,03-0000048361,001,002,999,001,288,1,0,007,,...,1,002,,2008-0201-006,03-048361,88934,,202976.86616,542798.94598,
4,03-0000015050,001,002,999,001,302,1,0,007,,...,1,002,,2008-0201-006,03-015050,88935,,203026.11371,542800.32686,


In [20]:
df.columns

Index(['부착대관리번호', '상태 (공통)', '부착방식', '부착대길이', '고가 (공통)', '부착대방향', '신호등수량',
       '배면등수량', '신호등종류', '배면등종류', '설치일', '교체일', '지주관리번호', '신호발광구분', '제조회사',
       '작업구분 (공통)', '표출구분 (공통)', '신규정규화ID', '공사관리번호', '부착대관리번호.1', '이력ID',
       '위치정보', 'X좌표', 'Y좌표', '공사형태 (공통)'],
      dtype='object')

In [21]:
df = df.drop(['상태 (공통)', '부착방식', '부착대길이', '고가 (공통)', '부착대방향', '배면등수량',
       '신호등종류', '배면등종류', '설치일', '교체일', '지주관리번호', '신호발광구분', '제조회사', '작업구분 (공통)',
       '표출구분 (공통)', '신규정규화ID', '공사관리번호', '부착대관리번호.1', '이력ID', '위치정보',
       '공사형태 (공통)'],axis='columns')

In [22]:
df.head(5)

,부착대관리번호,신호등수량,X좌표,Y좌표
0,03-0000034286,2,202650.00560,542705.46253
1,03-0000044336,3,202714.48048,542715.77569
2,03-0000044307,1,203129.34784,541838.75856
3,03-0000048361,1,202976.86616,542798.94598
4,03-0000015050,1,203026.11371,542800.32686


In [26]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString


df['X좌표'] = df['X좌표'].astype(float)
df['Y좌표'] = df['Y좌표'].astype(float)
df['geometry'] = df.apply(lambda row : Point([row['Y좌표'], row['X좌표']]), axis=1)
df = gpd.GeoDataFrame(df, geometry='geometry')

In [27]:
from fiona.crs import from_string

epsg5186 = from_string("+proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")

In [28]:
df.crs = epsg5186
print(df.crs)

+proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs +type=crs


In [29]:
wgs84 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [30]:
df = df.to_crs(wgs84)
df["geometry"].head()

0    POINT (130.72448 34.36235)
1    POINT (130.72462 34.36293)
2    POINT (130.71527 34.36695)
3    POINT (130.72562 34.36526)
4    POINT (130.72566 34.36571)
Name: geometry, dtype: geometry

In [31]:
df.head()

,부착대관리번호,신호등수량,X좌표,Y좌표,geometry
0,03-0000034286,2,202650.00560,542705.46253,POINT (130.72448 34.36235)
1,03-0000044336,3,202714.48048,542715.77569,POINT (130.72462 34.36293)
2,03-0000044307,1,203129.34784,541838.75856,POINT (130.71527 34.36695)
3,03-0000048361,1,202976.86616,542798.94598,POINT (130.72562 34.36526)
4,03-0000015050,1,203026.11371,542800.32686,POINT (130.72566 34.36571)


In [33]:
df.drop(['부착대관리번호','X좌표','Y좌표'],axis='columns')

,신호등수량,geometry
0,2,POINT (130.72448 34.36235)
1,3,POINT (130.72462 34.36293)
2,1,POINT (130.71527 34.36695)
3,1,POINT (130.72562 34.36526)
4,1,POINT (130.72566 34.36571)
...,...,...
109995,1,POINT (130.72129 34.19386)
109996,1,POINT (130.90334 34.38871)
109997,1,POINT (130.95081 34.37698)
109998,1,POINT (130.95104 34.37769)


In [34]:
df.dtypes

부착대관리번호       object
신호등수량          int64
X좌표          float64
Y좌표          float64
geometry    geometry
dtype: object

In [36]:
jongkak = [(37.572440, 126.979250),(37.570141, 126.979295),(37.569930, 126.982968),(37.572014, 126.983110)]
jongkak_poly = Polygon(jongkak)

In [44]:
test_code1 = Point(37.571233, 126.980684)
test_code1.within(jongkak_poly)

True

In [59]:
jongro = [(37.579836, 126.962647),(37.560608, 126.962958),(37.560526, 127.002665),(37.583205, 127.005879)]
jongro_poly = Polygon(jongro)

In [60]:
df['within'] = df['geometry'].within(jongro_poly)
df

,부착대관리번호,신호등수량,X좌표,Y좌표,geometry,within
0,03-0000034286,2,202650.00560,542705.46253,POINT (130.72448 34.36235),False
1,03-0000044336,3,202714.48048,542715.77569,POINT (130.72462 34.36293),False
2,03-0000044307,1,203129.34784,541838.75856,POINT (130.71527 34.36695),False
3,03-0000048361,1,202976.86616,542798.94598,POINT (130.72562 34.36526),False
4,03-0000015050,1,203026.11371,542800.32686,POINT (130.72566 34.36571),False
...,...,...,...,...,...,...
109995,03-0000076299,1,183933.70795,543097.23710,POINT (130.72129 34.19386),False
109996,03-0000036778,1,206196.04866,559059.56161,POINT (130.90334 34.38871),False
109997,03-0000037225,1,205062.84369,563479.41770,POINT (130.95081 34.37698),False
109998,03-0000037202,1,205142.04368,563497.91769,POINT (130.95104 34.37769),False


In [71]:
is_within = df['within'] == True
within_true = df[is_within]
within_true

,부착대관리번호,신호등수량,X좌표,Y좌표,geometry,within
